<a href="https://colab.research.google.com/github/suhaodatascichem/dashboard-streamlit/blob/main/aminoinsights_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

ModuleNotFoundError: No module named 'streamlit'

In [ ]:
# App title
st.title("Customer Data Dashboard")

# File uploader (customers upload their Excel files)
uploaded_files = st.file_uploader("Upload Excel file(s)", type=["xlsx"], accept_multiple_files=True)

if uploaded_files:
    st.write(f"📂 {len(uploaded_files)} file(s) uploaded.")

    # Read and process all uploaded files
    dataframes = []
    for file in uploaded_files:
        df = pd.read_excel(file)  # Read Excel file
        df["Source File"] = file.name  # Track which file it came from
        dataframes.append(df)

    # Merge all data into one DataFrame
    if dataframes:
        df = pd.concat(dataframes, ignore_index=True)
        st.write("### Processed Data Preview")
        st.dataframe(df)

        # Show data summary
        st.write(f"✅ Total Records: {df.shape[0]}")


In [ ]:
# Sidebar options
st.sidebar.header("Filters")

# Customer filter
customers = df["Customer"].unique()
selected_customer = st.sidebar.multiselect("Select Customer(s)", customers, default=customers)

# Material Type filter
material_types = df["Material"].unique()
selected_material = st.sidebar.multiselect("Select Material Type(s)", material_types, default=material_types)

# Date range filter
min_date = df["Release Date"].min()
max_date = df["Release Date"].max()
start_date, end_date = st.sidebar.date_input("Select Date Range", [min_date, max_date], min_value=min_date, max_value=max_date)

# Convert date inputs to datetime
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Reset filters button
if st.sidebar.button("Reset Filters"):
    selected_customer = customers
    selected_material = material_types
    start_date = min_date
    end_date = max_date
    st.sidebar.experimental_rerun()  # This forces the app to rerun with default filters

# Apply filters
filtered_df = df[
    (df["Customer"].isin(selected_customer)) &
    (df["Material"].isin(selected_material)) &
    (df["Release Date"] >= start_date) &
    (df["Release Date"] <= end_date)
]

# Filter the dataframe
filtered_df = df[df["Customer"].isin(selected_customer) & df["Material"].isin(selected_material)]

# Display count of rows after filtering
st.write(f"🔍 **Total Records After Filtering:** {filtered_df.shape[0]}")

st.dataframe(filtered_df)

2025-03-20 12:12:01.212 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 12:12:01.438 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-20 12:12:01.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 12:12:01.442 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 12:12:01.443 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 12:12:01.444 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 12:12:01.445 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-20 12:12:01.446 Session state does not 

In [ ]:
# create Visualization
chart_type = st.sidebar.selectbox("Select Chart Type", ["Scatter Plot", "Bar Chart"])

# Column selection
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()

if len(numeric_columns) < 2:
    st.error("Dataset must contain at least two numeric columns for visualization.")
else:
    x_axis = st.sidebar.selectbox("Select X-Axis", numeric_columns)
    y_axis = st.sidebar.selectbox("Select Y-Axis", numeric_columns)

    # Generate chart
    st.title("📊 Data Visualization Dashboard")

    if chart_type == "Scatter Plot":
        fig = px.scatter(df, x=x_axis, y=y_axis, title=f"Scatter Plot: {x_axis} vs {y_axis}")
        st.plotly_chart(fig, use_container_width=True)

    elif chart_type == "Bar Chart":
        fig = px.bar(df, x=x_axis, y=y_axis, title=f"Bar Chart: {x_axis} vs {y_axis}")
        st.plotly_chart(fig, use_container_width=True)

    # Show dataset preview
    with st.expander("🔍 View Dataset"):
        st.write(df.head())